In [28]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
import six
import sys
sys.modules['sklearn.externals.six'] = six
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus
import graphviz
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

Importing data

In [29]:
df = pd.read_excel("Absenteeism_at_work.xls")
df.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,97,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,97,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,97,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,97,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,97,0,1,2,1,0,1,90,172,30,2


In [30]:
df.drop(['ID','Month of absence','Service time','Hit target','Weight','Height'],axis='columns',inplace=True)
df.head()

,Reason for absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Age,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Body mass index,Absenteeism time in hours
0,26,3,1,289,36,33,0,1,2,1,0,1,30,4
1,0,3,1,118,13,50,1,1,1,1,0,0,31,0
2,23,4,1,179,51,38,0,1,0,1,0,0,31,2
3,7,5,1,279,5,39,0,1,2,1,1,0,24,4
4,23,5,1,289,36,33,0,1,2,1,0,1,30,2


In [31]:
df['Absenteeism time in hours'].replace([0],0,inplace=True)#"Parfait"
df['Absenteeism time in hours'].replace([1,2,3,4,5,6,7,8,9,10,16],1,inplace=True)#"Faible"
df['Absenteeism time in hours'].replace([24,40,32,56,48],2,inplace=True)#"Moyenne"
df['Absenteeism time in hours'].replace([120,80,64,112,104],3,inplace=True)#"Elever"
df['Absenteeism time in hours'].value_counts()

1    652
0     44
2     32
3     12
Name: Absenteeism time in hours, dtype: int64

In [32]:
df['Body mass index'].replace([24,23,19,22,21],1,inplace=True)#"Normal Weight"
df['Body mass index'].replace([25,28,27,29],2,inplace=True)#"Over Weight"
df['Body mass index'].replace([31,30,32,38,33,35,34,36],3,inplace=True)#"Obesityt"
df['Body mass index'].value_counts()

1    264
3    244
2    232
Name: Body mass index, dtype: int64

In [33]:
df['Age'].replace([28,33,30,31,32,29,27],1,inplace=True)#"[25-35]"
df['Age'].replace([38,37,40,36,41,34,43,39],2,inplace=True)#"[36-45]"
df['Age'].replace([50,47,58,48,49,46,53],3,inplace=True)#"[45:]"
df['Age'].value_counts()

2    394
1    263
3     83
Name: Age, dtype: int64

In [34]:
df['Distance from Residence to Work'].replace([10,11,12,13,14,15,16,17,5,20],1,inplace=True)#"proche "
df['Distance from Residence to Work'].replace([26,25,36,31,22,29,27,35],2,inplace=True)#"Normale"
df['Distance from Residence to Work'].replace([51,50,52,49,42,48,45],3,inplace=True)#"loin"
df['Distance from Residence to Work'].value_counts()

2    302
1    228
3    210
Name: Distance from Residence to Work, dtype: int64

In [35]:
df['Transportation expense'].replace([179,118,155,189,184,157],1,inplace=True)#"pas cher "
df['Transportation expense'].replace([255,235,289,291,246,248,228,233,279,268,231,225,260],2,inplace=True)#"abordable"
df['Transportation expense'].replace([361,330,369,378,300,388],3,inplace=True)#"cher "
df['Transportation expense'].value_counts()

2    346
1    323
3     71
Name: Transportation expense, dtype: int64

In [36]:
df['Reason for absence'].replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20,21,22,24,25,26],5,inplace=True)#"others "
df['Reason for absence'].replace(23,1,inplace=True)#"medical consultation  "
df['Reason for absence'].replace(28,2,inplace=True)#"dental consultation"
df['Reason for absence'].replace(27,3,inplace=True)#" physiotherapy  "
df['Reason for absence'].replace(14,4,inplace=True)#" Diseases of the genitourinary system  "
df['Reason for absence'].value_counts()

5    391
1    149
2    112
3     69
4     19
Name: Reason for absence, dtype: int64

In [37]:
df.rename(columns={"Absenteeism time in hours": "target"}, inplace=True)

In [38]:
y=df['target'] #  determine le target 
X= df.iloc[:,:-1]

In [39]:
y

0      1
1      0
2      1
3      1
4      1
5      1
6      1
7      1
8      2
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     2
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
710    1
711    2
712    1
713    1
714    0
715    0
716    1
717    1
718    1
719    1
720    1
721    1
722    1
723    1
724    1
725    1
726    1
727    1
728    1
729    3
730    1
731    1
732    1
733    1
734    3
735    1
736    1
737    0
738    0
739    0
Name: target, Length: 740, dtype: int64

In [40]:
model =  DecisionTreeClassifier()

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=5)# diviser le data set pour train le model
print(X_train.shape)# les données utilisé pour le training
print(X_test.shape)#les données utilisé pour le test
df.shape

(592, 13)
(148, 13)


(740, 14)

In [42]:
model.fit(X_train,y_train)#train le modél

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [43]:
model.score(X_test,y_test)# evaluer le modél

0.8716216216216216

In [44]:
predict_train = model.predict(X_train)
predict_test = model.predict(X_test)

In [45]:
#cross_val_score(DecisionTreeClassifier(),X_train,y_train,cv=5,scoring='accuracy').mean()

In [46]:
dot_data = StringIO()

In [47]:
export_graphviz(model,out_file=dot_data,filled=True,rounded=True,special_characters=True)

In [48]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())

In [49]:
graph.write_png('C4.5.png')

True